In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
import tensorflow as tf
import binascii
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

In [2]:
import json
import csv

with open('./data/json-data/pcap-wordpress1.json') as json_file:
    jsondata = json.load(json_file)

In [3]:
data_file = open('./data/pcap.csv', 'w', newline='')
csv_writer = csv.writer(data_file)

list_of_headers = []
 
csv_writer.writerow(jsondata[0].keys())

for data in jsondata:
    csv_writer.writerow(data.values())
    
data_file.close()

In [4]:
csv_data = pd.read_csv('./data/pcap.csv')

In [5]:
csv_data

,eth.dst,eth.src,eth.type,ip.dst,ip.flags,ip.ihl,ip.len,ip.proto,ip.src,ip.tos,ip.ttl,ip.version,length,load,load.count,protocol,protocol.dport,protocol.sport
0,a6:4d:66:07:0e:f8,16:78:bc:af:56:36,2048,10.244.0.11,DF,5.0,82.0,6.0,10.240.0.4,0.0,64.0,4.0,96,17 03 03 00 19 f5 9c c1,30.0,TCP,49190.0,10250.0
1,a6:4d:66:07:0e:f8,16:78:bc:af:56:36,2048,10.244.0.11,DF,5.0,158.0,6.0,10.240.0.4,0.0,64.0,4.0,172,17 03 03 00 65 7a 07 8b,106.0,TCP,49190.0,10250.0
2,16:78:bc:af:56:36,a6:4d:66:07:0e:f8,2048,52.143.28.136,DF,5.0,216.0,6.0,10.244.0.11,16.0,64.0,4.0,230,b9 4f 72 cb 17 e2 e0 a6,164.0,TCP,9000.0,54822.0
3,a6:4d:66:07:0e:f8,16:78:bc:af:56:36,2048,10.244.0.11,DF,5.0,52.0,6.0,52.143.28.136,0.0,62.0,4.0,66,NaN,NaN,TCP,54822.0,9000.0
4,a6:4d:66:07:0e:f8,16:78:bc:af:56:36,2048,10.244.0.11,DF,5.0,184.0,6.0,52.143.28.136,0.0,62.0,4.0,198,25 e8 57 a5 05 11 dc b7,132.0,TCP,54822.0,9000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43123,1a:37:6a:88:60:f6,16:78:bc:af:56:36,2048,10.244.0.6,DF,5.0,52.0,6.0,10.244.0.2,0.0,63.0,4.0,66,NaN,NaN,TCP,25227.0,47022.0
43124,1a:37:6a:88:60:f6,42:1b:a1:91:67:1c,2048,10.244.0.6,DF,5.0,52.0,6.0,10.244.0.2,0.0,64.0,4.0,66,NaN,NaN,TCP,25227.0,47022.0
43125,1a:37:6a:88:60:f6,16:78:bc:af:56:36,2048,10.244.0.6,DF,5.0,52.0,6.0,10.244.0.2,0.0,63.0,4.0,66,NaN,NaN,TCP,25227.0,47022.0
43126,42:1b:a1:91:67:1c,1a:37:6a:88:60:f6,2048,10.244.0.2,DF,5.0,64.0,6.0,10.244.0.6,0.0,64.0,4.0,78,NaN,NaN,TCP,47022.0,25227.0


In [6]:
csv_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43128 entries, 0 to 43127
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   eth.dst         43128 non-null  object 
 1   eth.src         43128 non-null  object 
 2   eth.type        43128 non-null  int64  
 3   ip.dst          43088 non-null  object 
 4   ip.flags        42614 non-null  object 
 5   ip.ihl          43088 non-null  float64
 6   ip.len          43088 non-null  float64
 7   ip.proto        43088 non-null  float64
 8   ip.src          43088 non-null  object 
 9   ip.tos          43088 non-null  float64
 10  ip.ttl          43088 non-null  float64
 11  ip.version      43088 non-null  float64
 12  length          43128 non-null  int64  
 13  load            18641 non-null  object 
 14  load.count      18641 non-null  float64
 15  protocol        42860 non-null  object 
 16  protocol.dport  42860 non-null  float64
 17  protocol.sport  42860 non-null 

In [7]:
csv_data.columns

Index(['eth.dst', 'eth.src', 'eth.type', 'ip.dst', 'ip.flags', 'ip.ihl',
       'ip.len', 'ip.proto', 'ip.src', 'ip.tos', 'ip.ttl', 'ip.version',
       'length', 'load', 'load.count', 'protocol', 'protocol.dport',
       'protocol.sport'],
      dtype='object')

## Ethernet headers

In [8]:
set(csv_data['eth.dst'])

{'0a:dc:5b:dc:e2:46',
 '0e:94:1c:e8:6b:c3',
 '16:78:bc:af:56:36',
 '1a:37:6a:88:60:f6',
 '22:a4:08:cd:09:59',
 '26:9c:42:81:1e:3f',
 '36:37:b8:5e:c0:ae',
 '42:1b:a1:91:67:1c',
 '7e:1a:07:d0:98:46',
 '8e:23:39:d7:c0:7b',
 'a6:4d:66:07:0e:f8',
 'b2:72:6c:1c:fc:c2'}

In [9]:
set(csv_data['eth.src'])

{'0a:dc:5b:dc:e2:46',
 '0e:94:1c:e8:6b:c3',
 '16:78:bc:af:56:36',
 '1a:37:6a:88:60:f6',
 '22:a4:08:cd:09:59',
 '26:9c:42:81:1e:3f',
 '36:37:b8:5e:c0:ae',
 '42:1b:a1:91:67:1c',
 '7e:1a:07:d0:98:46',
 '8e:23:39:d7:c0:7b',
 'a6:4d:66:07:0e:f8',
 'b2:72:6c:1c:fc:c2'}

In [10]:
list_of_stuff = '0a:dc:5b:dc:e2:46'.split(':')

In [11]:
hex_lambda = lambda x: int(x,16)

In [12]:
list(map(hex_lambda, list_of_stuff))

[10, 220, 91, 220, 226, 70]

In [13]:
set(csv_data['eth.type'])

{2048, 2054}

## IP headers

In [14]:
set(csv_data['ip.dst']) # split them on . and store as new number

{'10.240.0.4',
 '10.244.0.1',
 '10.244.0.10',
 '10.244.0.11',
 '10.244.0.13',
 '10.244.0.14',
 '10.244.0.2',
 '10.244.0.3',
 '10.244.0.4',
 '10.244.0.5',
 '10.244.0.6',
 '10.244.0.7',
 '10.244.0.9',
 '13.69.106.216',
 '13.69.106.90',
 '13.69.114.0',
 '13.69.65.27',
 '13.69.65.29',
 '13.69.65.31',
 '13.69.67.61',
 '168.63.129.16',
 '169.254.169.254',
 '198.143.164.251',
 '20.50.65.82',
 '40.113.176.130',
 '52.143.28.136',
 '52.236.186.216',
 '52.236.186.217',
 '89.100.107.147',
 nan}

In [15]:
print(set(csv_data['ip.src']))

{nan, '10.244.0.14', '13.69.65.31', '52.236.186.217', '10.244.0.11', '20.50.65.82', '10.244.0.6', '52.143.28.136', '13.69.65.29', '10.244.0.2', '10.244.0.9', '10.244.0.13', '13.69.106.216', '168.63.129.16', '10.240.0.4', '13.69.65.27', '13.69.106.90', '10.244.0.7', '198.143.164.251', '40.113.176.130', '10.244.0.3', '13.69.114.0', '52.236.186.216', '10.244.0.4', '10.244.0.10', '13.69.67.61', '10.244.0.5', '169.254.169.254', '10.244.0.1'}


In [16]:
print(set(csv_data['ip.flags'])) #turn nan to zero? 

{nan, 'DF'}


In [17]:
print(set(csv_data['ip.ihl'])) ## remove this, most are Nan 

{nan, nan, nan, nan, nan, 5.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan}


In [18]:
print(set(csv_data['ip.len'])) 

{nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 2060.0, 20493.0, nan, nan, 8204.0, 4111.0, 2066.0, 2076.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, 40.0, 4136.0, 2099.0, 52.0, 16436.0, 12340.0, 2101.0, 4148.0, 57.0, 55.0, 6203.0, 60.0, 61.0, 6204.0, 63.0, 64.0, 2048.0, 4154.0, 69.0, 71.0, 2120.0, 74.0, 4170.0, 76.0, 77.0, 78.0, 6223.0, 80.0, 81.0, 82.0, 4179.0, 83.0, 85.0, 86.0, 87.0, 88.0, 84.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 6241.0, 98.0, 24668.0, 100.0, 2142.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 4204.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 4211.0, 116.0, 115.0, 118.0, 119.0, 120.0, 4214.0, 122.0, 123.0, 124.0, 125.0, 10366.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 156.0, 157.0, 158.0, 159.0, 160.0, 161.0, 162.0, 163.0, nan, 165.0, 166.0, 167.0, 168.0, 169.0, 170.0, 171.0, 172.0, 173.0, 174

In [19]:
print(set(csv_data['ip.proto']))

{nan, nan, 1.0, nan, nan, nan, 6.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 17.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan}


In [20]:
#', '', '', '
print(set(csv_data['ip.tos']))

{0.0, nan, 2.0, nan, nan, nan, nan, nan, 8.0, nan, nan, nan, nan, nan, nan, nan, 16.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 192.0, nan, nan, nan, nan, nan, nan, nan, nan}


In [21]:
print(set(csv_data['ip.ttl']))

{nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 55.0, 62.0, 63.0, 64.0, nan, nan, nan, nan, 114.0, 115.0, 120.0, 122.0, 127.0}


In [22]:
print(set(csv_data['ip.version']))

{nan, nan, nan, nan, 4.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan}


## general and load
 13  load          18641 non-null  object 
 14  load.count    18641 non-null  float64

In [23]:
print(set(csv_data['length']))

{14346, 6157, 2062, 2066, 2074, 20507, 8218, 4125, 2080, 42, 2090, 54, 4150, 2113, 66, 16450, 12354, 2115, 4162, 71, 69, 6217, 74, 75, 6218, 77, 78, 4168, 83, 85, 2134, 88, 4184, 90, 91, 92, 6237, 94, 95, 96, 4193, 97, 99, 100, 101, 102, 98, 104, 105, 106, 107, 108, 109, 110, 6255, 112, 24682, 114, 2156, 116, 117, 118, 119, 120, 121, 4218, 123, 124, 125, 126, 127, 128, 4225, 130, 129, 132, 133, 134, 4228, 136, 137, 138, 135, 10380, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 170, 171, 172, 173, 174, 175, 176, 177, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 2234, 193, 194, 195, 196, 197, 198, 8390, 200, 199, 202, 203, 201, 205, 206, 207, 2255, 208, 14546, 211, 212, 213, 214, 2259, 210, 217, 218, 2258, 2269, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 8424, 242, 4334, 244, 245, 246, 250, 251, 253, 4350, 255, 256, 257, 254

In [24]:
print(set(csv_data['load'])) 

{nan, '28 0a ff be 23 6a 49 f6', 'fa 6d c0 07 c4 52 7b 28', '17 03 03 29 c3 9a f4 bc', '17 03 03 02 5a 1e 24 64', '17 03 03 00 a3 94 ba a9', '98 09 bb c5 ac 77 5a 7a', '17 03 03 00 13 e9 96 78', 'df f6 fe a3 f4 c5 d0 ed', 'a7 39 8f 95 30 b4 5e 15', '31 30 36 3a 22 2f 6f 70', '0a 0f 6c 2b 01 4e 3c cc', '16 fa ba dd 03 36 db b6', '17 03 03 00 a8 a4 0e 0c', '02 94 fa 4f b2 5b 20 a2', 'e7 38 19 a5 85 65 a7 8f', '17 03 03 04 b3 79 60 92', '17 03 03 00 15 03 b1 59', 'c8 00 00 00 03 53 45 4c', '17 03 03 01 3e 6e 45 9b', '8a a0 ad 80 a9 bf 7c cc', '17 03 03 01 31 e7 6a 1a', '3f 6b 85 aa 99 42 46 2e', '48 00 00 00 03 53 45 4c', '44 ab 3e f5 9c 7e 28 cd', '17 03 03 00 a8 d1 71 f6', 'e8 b1 67 74 19 26 57 47', '17 03 03 00 3e 33 b0 10', '17 03 03 00 22 d6 ee 32', '6d 25 f4 45 b3 06 11 df', '68 b8 17 ee 23 7a d1 e2', '17 03 03 00 13 4e 79 cb', '17 03 03 00 51 5b 6d e9', '39 00 00 00 03 55 50 44', '26 0f a7 57 fd 63 7e 5c', '51 b9 9a e9 fc 8c 52 bd', 'f0 46 d1 d7 89 b9 61 78', '24 9e 78 dd 43 06 64 

In [25]:
print(set(csv_data['load.count']))

{nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 11.0, 9.0, 3.0, 22.0, 24.0, 26.0, 5.0, 28.0, 29.0, 30.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 36.0, 38.0, 42.0, 39.0, 41.0, 40.0, 44.0, 46.0, 48.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 49.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 104.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 134.0, 135.0, 136.0, 137.0, 139.0, 140.0, 141.0, 142.0, 144.0, 145.0, 146.0, 147.0, 148.0, 151.0, 152.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 164.0, 161.0, 33.0, 168.0, 170.0, 32.0, 34.0, 173.0, 172.0, 169.0, 176.0, 35.0, 178.0, 171.0, 180.0, 179.0, 188.

## Protocols

In [26]:
# tcp.dport     40140 non-null  float64
#  16  tcp.flags     40140 non-null  object 
#  17  tcp.reserved  40140 non-null  float64
#  18  tcp.sport     40140 non-null  float64
#  19  tcp.urgptr    40140 non-null  float64
#  20  udp.dport     2720 non-null   float64
#  21  udp.len       2720 non-null   float64
#  22  udp.sport     2720 non-null   float64
csv_data[csv_data['protocol'].isna()]

,eth.dst,eth.src,eth.type,ip.dst,ip.flags,ip.ihl,ip.len,ip.proto,ip.src,ip.tos,ip.ttl,ip.version,length,load,load.count,protocol,protocol.dport,protocol.sport
244,8e:23:39:d7:c0:7b,42:1b:a1:91:67:1c,2048,10.244.0.10,NaN,5.0,360.0,1.0,10.244.0.2,192.0,64.0,4.0,374,NaN,NaN,NaN,NaN,NaN
245,8e:23:39:d7:c0:7b,16:78:bc:af:56:36,2048,10.244.0.10,NaN,5.0,360.0,1.0,10.244.0.2,192.0,63.0,4.0,374,NaN,NaN,NaN,NaN,NaN
295,b2:72:6c:1c:fc:c2,42:1b:a1:91:67:1c,2048,10.244.0.3,NaN,5.0,360.0,1.0,10.244.0.2,192.0,64.0,4.0,374,NaN,NaN,NaN,NaN,NaN
296,b2:72:6c:1c:fc:c2,16:78:bc:af:56:36,2048,10.244.0.3,NaN,5.0,360.0,1.0,10.244.0.2,192.0,63.0,4.0,374,NaN,NaN,NaN,NaN,NaN
358,0a:dc:5b:dc:e2:46,b2:72:6c:1c:fc:c2,2054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41915,8e:23:39:d7:c0:7b,16:78:bc:af:56:36,2048,10.244.0.10,NaN,5.0,360.0,1.0,10.244.0.2,192.0,63.0,4.0,374,NaN,NaN,NaN,NaN,NaN
42626,36:37:b8:5e:c0:ae,16:78:bc:af:56:36,2054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42,NaN,NaN,NaN,NaN,NaN
42627,16:78:bc:af:56:36,36:37:b8:5e:c0:ae,2054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42,NaN,NaN,NaN,NaN,NaN
43114,42:1b:a1:91:67:1c,8e:23:39:d7:c0:7b,2054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42,NaN,NaN,NaN,NaN,NaN


In [27]:
csv_data[csv_data['protocol.dport'].isna()] #fill na with 0? 

,eth.dst,eth.src,eth.type,ip.dst,ip.flags,ip.ihl,ip.len,ip.proto,ip.src,ip.tos,ip.ttl,ip.version,length,load,load.count,protocol,protocol.dport,protocol.sport
244,8e:23:39:d7:c0:7b,42:1b:a1:91:67:1c,2048,10.244.0.10,NaN,5.0,360.0,1.0,10.244.0.2,192.0,64.0,4.0,374,NaN,NaN,NaN,NaN,NaN
245,8e:23:39:d7:c0:7b,16:78:bc:af:56:36,2048,10.244.0.10,NaN,5.0,360.0,1.0,10.244.0.2,192.0,63.0,4.0,374,NaN,NaN,NaN,NaN,NaN
295,b2:72:6c:1c:fc:c2,42:1b:a1:91:67:1c,2048,10.244.0.3,NaN,5.0,360.0,1.0,10.244.0.2,192.0,64.0,4.0,374,NaN,NaN,NaN,NaN,NaN
296,b2:72:6c:1c:fc:c2,16:78:bc:af:56:36,2048,10.244.0.3,NaN,5.0,360.0,1.0,10.244.0.2,192.0,63.0,4.0,374,NaN,NaN,NaN,NaN,NaN
358,0a:dc:5b:dc:e2:46,b2:72:6c:1c:fc:c2,2054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41915,8e:23:39:d7:c0:7b,16:78:bc:af:56:36,2048,10.244.0.10,NaN,5.0,360.0,1.0,10.244.0.2,192.0,63.0,4.0,374,NaN,NaN,NaN,NaN,NaN
42626,36:37:b8:5e:c0:ae,16:78:bc:af:56:36,2054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42,NaN,NaN,NaN,NaN,NaN
42627,16:78:bc:af:56:36,36:37:b8:5e:c0:ae,2054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42,NaN,NaN,NaN,NaN,NaN
43114,42:1b:a1:91:67:1c,8e:23:39:d7:c0:7b,2054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42,NaN,NaN,NaN,NaN,NaN


In [28]:
#drop the columns you dont need
csv_data = csv_data[csv_data['protocol'].notna()] 
csv_data = csv_data.drop(['eth.type'], axis = 1) 

In [29]:
csv_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42860 entries, 0 to 43127
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   eth.dst         42860 non-null  object 
 1   eth.src         42860 non-null  object 
 2   ip.dst          42860 non-null  object 
 3   ip.flags        42614 non-null  object 
 4   ip.ihl          42860 non-null  float64
 5   ip.len          42860 non-null  float64
 6   ip.proto        42860 non-null  float64
 7   ip.src          42860 non-null  object 
 8   ip.tos          42860 non-null  float64
 9   ip.ttl          42860 non-null  float64
 10  ip.version      42860 non-null  float64
 11  length          42860 non-null  int64  
 12  load            18641 non-null  object 
 13  load.count      18641 non-null  float64
 14  protocol        42860 non-null  object 
 15  protocol.dport  42860 non-null  float64
 16  protocol.sport  42860 non-null  float64
dtypes: float64(9), int64(1), object

In [30]:
set(csv_data['ip.flags'])

{'DF', nan}

In [49]:
csv_data['load'].fillna(0.0, inplace = True)
csv_data['load.count'].fillna(0.0, inplace = True)
csv_data['ip.flags'].fillna("0", inplace = True)

In [32]:
csv_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42860 entries, 0 to 43127
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   eth.dst         42860 non-null  object 
 1   eth.src         42860 non-null  object 
 2   ip.dst          42860 non-null  object 
 3   ip.flags        42860 non-null  object 
 4   ip.ihl          42860 non-null  float64
 5   ip.len          42860 non-null  float64
 6   ip.proto        42860 non-null  float64
 7   ip.src          42860 non-null  object 
 8   ip.tos          42860 non-null  float64
 9   ip.ttl          42860 non-null  float64
 10  ip.version      42860 non-null  float64
 11  length          42860 non-null  int64  
 12  load            42860 non-null  object 
 13  load.count      42860 non-null  float64
 14  protocol        42860 non-null  object 
 15  protocol.dport  42860 non-null  float64
 16  protocol.sport  42860 non-null  float64
dtypes: float64(9), int64(1), object

In [33]:
#Plan 
# For ethernet: make different columns for octects, convert hex to int and normalize
hex_lambda = lambda x: int(x,16)

##eth_dst
eth_dst_list = list(csv_data['eth.dst'])

octet_1_dst = []
octet_2_dst = []
octet_3_dst = []
octet_4_dst = []
octet_5_dst = []
octet_6_dst = []

for eth in eth_dst_list:
    int_eth_list = list(map(hex_lambda, eth.split(':')))
    octet_1_dst.append(int_eth_list[0])
    octet_2_dst.append(int_eth_list[1])
    octet_3_dst.append(int_eth_list[2])
    octet_4_dst.append(int_eth_list[3])
    octet_5_dst.append(int_eth_list[4])
    octet_6_dst.append(int_eth_list[5])

final_df = pd.DataFrame(octet_1_dst, columns =['octet_1_eth_dst'])
final_df['octet_2_eth_dst'] = octet_2_dst
final_df['octet_3_eth_dst'] = octet_3_dst
final_df['octet_4_eth_dst'] = octet_4_dst
final_df['octet_5_eth_dst'] = octet_5_dst
final_df['octet_6_eth_dst'] = octet_6_dst

In [34]:
##eth_src
eth_src_list = list(csv_data['eth.src'])

octet_1_src = []
octet_2_src = []
octet_3_src = []
octet_4_src = []
octet_5_src = []
octet_6_src = []

for eth in eth_src_list:
    int_eth_list = list(map(hex_lambda, eth.split(':')))
    octet_1_src.append(int_eth_list[0])
    octet_2_src.append(int_eth_list[1])
    octet_3_src.append(int_eth_list[2])
    octet_4_src.append(int_eth_list[3])
    octet_5_src.append(int_eth_list[4])
    octet_6_src.append(int_eth_list[5])
    
final_df['octet_1_eth_src'] = octet_1_src 
final_df['octet_2_eth_src'] = octet_2_src
final_df['octet_3_eth_src'] = octet_3_src
final_df['octet_4_eth_src'] = octet_4_src
final_df['octet_5_eth_src'] = octet_5_src
final_df['octet_6_eth_src'] = octet_6_src


In [35]:
## ip.dst and ip.src

ip_src_list = list(csv_data['ip.src'])

octet_1_src = []
octet_2_src = []
octet_3_src = []
octet_4_src = []

for eth in ip_src_list:
    int_ip_list = list(eth.split('.'))
    octet_1_src.append(int_ip_list[0])
    octet_2_src.append(int_ip_list[1])
    octet_3_src.append(int_ip_list[2])
    octet_4_src.append(int_ip_list[3])

final_df['octet_1_ip_src'] = octet_1_src 
final_df['octet_2_ip_src'] = octet_2_src
final_df['octet_3_ip_src'] = octet_3_src
final_df['octet_4_ip_src'] = octet_4_src


In [36]:
## ip.dst and ip.src

ip_dst_list = list(csv_data['ip.dst'])

octet_1_dst = []
octet_2_dst = []
octet_3_dst = []
octet_4_dst = []

for eth in ip_dst_list:
    int_ip_list = list(eth.split('.'))
    octet_1_dst.append(int_ip_list[0])
    octet_2_dst.append(int_ip_list[1])
    octet_3_dst.append(int_ip_list[2])
    octet_4_dst.append(int_ip_list[3])

final_df['octet_1_ip_dst'] = octet_1_dst
final_df['octet_2_ip_dst'] = octet_2_dst
final_df['octet_3_ip_dst'] = octet_3_dst
final_df['octet_4_ip_dst'] = octet_4_dst

In [67]:
#  4   ip.flags        42860 non-null  object    DROP
#  5   ip.ihl          42860 non-null  float64   DROP
#  6   ip.len          42860 non-null  float64   DONE 
#  7   ip.proto        42860 non-null  float64   DROP
#  9   ip.tos          42860 non-null  float64   DONE
#  10  ip.ttl          42860 non-null  float64   DONE
#  11  ip.version   DROP

# drop ip version as all are 4
# drop ihl as everything is 5
# drop flags cause fuck it

set(csv_data['length'])

{54,
 66,
 69,
 71,
 74,
 75,
 77,
 78,
 83,
 85,
 88,
 90,
 91,
 92,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 112,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 205,
 206,
 207,
 208,
 210,
 211,
 212,
 213,
 214,
 217,
 218,
 222,
 223,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 242,
 244,
 245,
 246,
 250,
 253,
 254,
 257,
 260,
 262,
 265,
 266,
 270,
 272,
 274,
 276,
 277,
 278,
 282,
 283,
 284,
 285,
 286,
 288,


In [72]:
final_df['ip.len'] = list(csv_data['ip.len'])
final_df['ip.tos'] = list(csv_data['ip.tos'])
final_df['ip.ttl'] = list(csv_data['ip.ttl'])
final_df['length'] = list(csv_data['length'])
final_df['load.count'] = list(csv_data['load.count'])

In [75]:
dummy_protocol = pd.get_dummies(csv_data['protocol'])

In [79]:
final_df['TCP'] = list(dummy_protocol['TCP'])
final_df['UDP'] = list(dummy_protocol['UDP'])
final_df['protocol.dport'] = list(csv_data['protocol.dport'])
final_df['protocol.sport'] = list(csv_data['protocol.sport'])

In [86]:
final_df = final_df.astype('float64')

In [89]:
from sklearn.preprocessing import StandardScaler

num_cols = ['octet_1_eth_dst',
 'octet_2_eth_dst',
 'octet_3_eth_dst',
 'octet_4_eth_dst',
 'octet_5_eth_dst',
 'octet_6_eth_dst',
 'octet_1_eth_src',
 'octet_2_eth_src',
 'octet_3_eth_src',
 'octet_4_eth_src',
 'octet_5_eth_src',
 'octet_6_eth_src',
 'octet_1_ip_src',
 'octet_2_ip_src',
 'octet_3_ip_src',
 'octet_4_ip_src',
 'octet_1_ip_dst',
 'octet_2_ip_dst',
 'octet_3_ip_dst',
 'octet_4_ip_dst',
 'ip.len',
 'ip.tos',
 'ip.ttl',
 'length',
 'load.count',
 'protocol.dport',
 'protocol.sport']

for i in num_cols:
    scale = StandardScaler().fit(final_df[[i]])
    final_df[i] = scale.transform(final_df[[i]])


In [90]:
final_df

,octet_1_eth_dst,octet_2_eth_dst,octet_3_eth_dst,octet_4_eth_dst,octet_5_eth_dst,octet_6_eth_dst,octet_1_eth_src,octet_2_eth_src,octet_3_eth_src,octet_4_eth_src,...,octet_4_ip_dst,ip.len,ip.tos,ip.ttl,length,load.count,TCP,UDP,protocol.dport,protocol.sport
0,3.179631,-1.370416,0.333560,-3.360464,-1.860455,1.660922,-0.249040,-0.376649,1.297941,-0.219675,...,-0.308945,-0.220816,-0.700236,-0.053118,-0.220816,-0.217735,1.0,0.0,1.110736,-0.733448
1,3.179631,-1.370416,0.333560,-3.360464,-1.860455,1.660922,-0.249040,-0.376649,1.297941,-0.219675,...,-0.308945,-0.195166,-0.700236,-0.053118,-0.195166,-0.192100,1.0,0.0,1.110736,-0.733448
2,-0.171139,-0.619221,1.945418,-0.394295,-0.827133,-1.038847,2.971261,-1.244202,0.005060,-3.059419,...,3.278894,-0.175590,3.017125,-0.053118,-0.175590,-0.172537,1.0,0.0,-0.729208,1.284583
3,3.179631,-1.370416,0.333560,-3.360464,-1.860455,1.660922,-0.249040,-0.376649,1.297941,-0.219675,...,-0.308945,-0.230942,-0.700236,-0.286199,-0.230942,-0.227854,1.0,0.0,1.368575,-0.790043
4,3.179631,-1.370416,0.333560,-3.360464,-1.860455,1.660922,-0.249040,-0.376649,1.297941,-0.219675,...,-0.308945,-0.186391,-0.700236,-0.286199,-0.186391,-0.183330,1.0,0.0,1.368575,-0.790043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42855,-0.078062,-1.754748,0.408530,-1.082870,-0.683617,1.633089,-0.249040,-0.376649,1.297941,-0.219675,...,-0.452458,-0.230942,-0.700236,-0.169658,-0.230942,-0.227854,1.0,0.0,0.013682,0.931432
42856,-0.078062,-1.754748,0.408530,-1.082870,-0.683617,1.633089,0.734941,-2.252984,0.892037,-0.726772,...,-0.452458,-0.230942,-0.700236,-0.053118,-0.230942,-0.227854,1.0,0.0,0.013682,0.931432
42857,-0.078062,-1.754748,0.408530,-1.082870,-0.683617,1.633089,-0.249040,-0.376649,1.297941,-0.219675,...,-0.452458,-0.230942,-0.700236,-0.169658,-0.230942,-0.227854,1.0,0.0,0.013682,0.931432
42858,0.852707,-2.243899,1.439369,-0.923968,-0.583155,-1.400671,-0.159587,-1.688066,0.065194,-0.878902,...,-0.567269,-0.226891,-0.700236,-0.053118,-0.226891,-0.227854,1.0,0.0,1.011482,-0.055353


In [91]:
from sklearn.model_selection import train_test_split

x_normal_train, x_normal_test = train_test_split(
    final_df, test_size=0.25, random_state=42)